[https://docs.arbor-sim.org/en/stable/tutorial/single_cell_detailed.html](https://docs.arbor-sim.org/en/stable/tutorial/single_cell_detailed.html)

In [1]:
import arbor
from arbor import mpoint
from arbor import mnpos

In [2]:
# Define the morphology by manually building a segment tree
tree = arbor.segment_tree()

# Start with segment 0: a cylindrical soma with tag 1
tree.append(mnpos, mpoint(0.0, 0.0, 0.0, 2.0), mpoint( 40.0, 0.0, 0.0, 2.0), tag=1)
# Construct the first section of the dendritic tree with tag 3,
# comprised of segments 1 and 2, attached to soma segment 0.
tree.append(0,     mpoint(40.0, 0.0, 0.0, 0.8), mpoint( 80.0,  0.0, 0.0, 0.8), tag=3)
tree.append(1,     mpoint(80.0, 0.0, 0.0, 0.8), mpoint(120.0, -5.0, 0.0, 0.8), tag=3)
# Construct the rest of the dendritic tree: segments 3, 4 and 5.
tree.append(2,     mpoint(120.0, -5.0, 0.0, 0.8), mpoint(200.0,  40.0, 0.0, 0.4), tag=3)
tree.append(3,     mpoint(200.0, 40.0, 0.0, 0.4), mpoint(260.0,  60.0, 0.0, 0.2), tag=3)
tree.append(2,     mpoint(120.0, -5.0, 0.0, 0.5), mpoint(190.0, -30.0, 0.0, 0.5), tag=3)
# Construct a special region of the tree made of segments 6, 7, and 8
# differentiated from the rest of the tree using tag 4.
tree.append(5,     mpoint(190.0, -30.0, 0.0, 0.5), mpoint(240.0, -70.0, 0.0, 0.2), tag=4)
tree.append(5,     mpoint(190.0, -30.0, 0.0, 0.5), mpoint(230.0, -10.0, 0.0, 0.2), tag=4)
tree.append(7,     mpoint(230.0, -10.0, 0.0, 0.2), mpoint(360.0, -20.0, 0.0, 0.2), tag=4)
# Construct segments 9 and 10 that make up the axon with tag 2.
# Segment 9 is at the root, where its proximal end will be connected to the
# proximal end of the soma segment.
tree.append(mnpos, mpoint( 0.0, 0.0, 0.0, 2.0), mpoint(  -70.0, 0.0, 0.0, 0.4), tag=2)
tree.append(9,     mpoint(-70.0, 0.0, 0.0, 0.4), mpoint(-100.0, 0.0, 0.0, 0.4), tag=2)

morph = arbor.morphology(tree);

In [3]:
# (2) Create and populate the label dictionary.
labels = arbor.label_dict()

# Regions:

# Add labels for tag 1, 2, 3, 4
labels['soma'] = '(tag 1)'
labels['axon'] = '(tag 2)'
labels['dend'] = '(tag 3)'
labels['last'] = '(tag 4)'

# Add a label for a region that includes the whole morphology
labels['all'] = '(all)'
# Add a label for the parts of the morphology with radius greater than 1.5 μm.
labels['gt_1.5'] = '(radius-ge (region "all") 1.5)'

# Join regions "last" and "gt_1.5"
labels['custom'] = '(join (region "last") (region "gt_1.5"))'

# Add a labels for the root of the morphology and all the terminal points
labels['root']     = '(root)'
labels['terminal'] = '(terminal)'

# Add a label for the terminal locations in the "custom" region:
labels['custom_terminal'] = '(restrict (locset "terminal") (region "custom"))'
# Add a label for the terminal locations in the "axon" region:
labels['axon_terminal'] = '(restrict (locset "terminal") (region "axon"))'

In [4]:
# (3) Create and populate the decor.
decor = arbor.decor()

# Set the default properties of the cell (this overrides the model defaults).
decor.set_property(Vm =-55)
decor.set_ion('na', int_con=10,   ext_con=140, rev_pot=50, method='nernst/na')
decor.set_ion('k',  int_con=54.4, ext_con=2.5, rev_pot=-77)

In [5]:
# Override the cell defaults.
decor.paint('"custom"', tempK=270)
decor.paint('"soma"',   Vm=-50)

In [6]:
# Paint density mechanisms.
decor.paint('"all"', arbor.density('pas'))
decor.paint('"custom"', arbor.density('hh'))
decor.paint('"dend"',  arbor.density('Ih', {'gbar': 0.001}))

In [7]:
# Place stimuli and spike detectors.
decor.place('"root"', arbor.iclamp(10, 1, current=2), 'iclamp0')
decor.place('"root"', arbor.iclamp(30, 1, current=2), 'iclamp1')
decor.place('"root"', arbor.iclamp(50, 1, current=2), 'iclamp2')
decor.place('"axon_terminal"', arbor.spike_detector(-10), 'detector')

In [8]:
# Single CV for the "soma" region
soma_policy = arbor.cv_policy_single('"soma"')
# Single CV for the "soma" region
dflt_policy = arbor.cv_policy_max_extent(1.0)
# default policy everywhere except the soma
policy = dflt_policy | soma_policy
# Set cv_policy
decor.discretization(policy)

In [9]:
# (4) Create the cell.
cell = arbor.cable_cell(morph, labels, decor)

In [10]:
# (5) Construct the model
model = arbor.single_cell_model(cell)

In [11]:
# (6) Set the model default properties
model.properties.set_property(Vm =-65, tempK=300, rL=35.4, cm=0.01)
model.properties.set_ion('na', int_con=10,   ext_con=140, rev_pot=50, method='nernst/na')
model.properties.set_ion('k',  int_con=54.4, ext_con=2.5, rev_pot=-77)

In [12]:
# Extend the default catalogue with the Allen catalogue.
# The function takes a second string parameter that can prefix
# the name of the mechanisms to avoid collisions between catalogues
# in this case we have no collisions so we use an empty prefix string.
model.properties.catalogue.extend(arbor.allen_catalogue(), "")

In [13]:
# (7) Add probes.
# Add voltage probes on the "custom_terminal" locset
# which sample the voltage at 50 kHz
model.probe('voltage', where='"custom_terminal"',  frequency=50)

In [14]:
# (8) Run the simulation for 100 ms, with a dt of 0.025 ms
model.run(tfinal=100, dt=0.025)

In [15]:
# (9) Print the spikes.
print(len(model.spikes), 'spikes recorded:')
for s in model.spikes:
    print(s)

3 spikes recorded:
11.475749222188028
32.01355632346195
52.13541998326556
